In [1]:
import json
import os
import re

from ollama import chat
from ollama import ChatResponse

In [2]:
MODEL_NAME = 'phi4:14b'

# Create a list of agents

AGENTS
1. chat_agent
2. router_agent
3. research_agent
4. final_answer_agent

TOOLS
- Function that returns an avaerage dog weight given a dog breed
- Calculate function that returns answers to calculations

BLOCKS
- ReAct block

# Helper functions

In [3]:
def initialize_message_history(system_message):

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        }
                    ]

    return message_history

In [4]:
def create_message_history(system_message, user_input):

    message_history = [
                        {
                            "role": "system",
                            "content": system_message
                        },
                        {
                            "role": "user",
                            "content": user_input
                        }
                    ]

    return message_history



# Set up the LLM

In [5]:
def make_llm_api_call(message_history):

    model_name = MODEL_NAME

    response: ChatResponse = chat(model=model_name, 
                                  messages=message_history,
                                )

    output_text = response['message']['content']

    #thinking_text, response_text = process_response(output_text)

    return output_text


# Example

system_message = "Your name is Molly."
user_message = "What's your name?"

message_history = create_message_history(system_message, user_message)

response_text = make_llm_api_call(message_history)

print(response_text)

#print(response['message']['content'])

My name is Molly! How can I assist you today?


# Set up the tools



In [6]:
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return("Scottish Terriers average 20 lbs")
    elif name in "Border Collie":
        return("a Border Collies average weight is 37 lbs")
    elif name in "Toy Poodle":
        return("a toy poodles average weight is 7 lbs")
    else:
        return("An average dog weights 50 lbs")

known_actions = {
    "calculate": calculate,
    "average_dog_weight": average_dog_weight
}

# Set up the system messages

In [7]:
chat_agent_system_message = """
1. You provide polite answers to simple questions.
If the user's input requires only a simple answer, then output your answer as JSON.

Example session:

Question: Hello. How are you?

You output:

{{
"Answer": "I'm fine, thanks.",
"Status": "DONE"
}}

2. You also run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.
Output your response as a JSON string.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

You can only call one action at a time.


Example session:

Question: How much does a Bulldog weigh?
{
    "Thought": "I should look the dogs weight using average_dog_weight",
    "Action": {"function":"average_dog_weight", "input": "Bulldog"},
    "Status": "PAUSE"
}

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:
{
    "Answer": "A bulldog weights 51 lbs",
    "Status": "DONE"
}
""".strip()



# Set up the Agents

In [8]:
def run_chat_agent(message_history):

    print("---CHAT AGENT---")

    # Prompt the llm
    response = make_llm_api_call(message_history)

    response = response.replace('```json', '')
    response = response.replace('```', '')
    response = response.strip()

    print(response)

    return response



# Example

user_query = "How much does a toy poodle weigh?"

message_history = create_message_history(chat_agent_system_message, user_query)

# Prompt the chat_agent
response = run_chat_agent(message_history)

# Update message history
message = [{"role": "assistant", "content": response}]
message_history.append(message)


---CHAT AGENT---
{
    "Thought": "I should look up the average weight of a Toy Poodle using the average_dog_weight function.",
    "Action": {"function":"average_dog_weight", "input": "Toy Poodle"},
    "Status": "PAUSE"
}


In [9]:
def run_router_agent(llm_response):

    """
    Route to web search or not.
    Args:
        state (dict): The current graph state
    Returns:
        str: Next node to call
    """

    print("---ROUTER AGENT---")


    # Extract the status
    json_response = json.loads(llm_response)
    status = json_response['Status']
    #status = extract_json_str_value(response, "Status").strip()

    print("Status:", status)

    if status == 'PAUSE':
        print("Route: to_research_agent")
        return "to_research_agent"

    elif status == 'DONE':
        print("Route: to_final_answer")
        return "to_final_answer"



# Example

user_query = "How much does a toy poodle weigh?"

message_history = create_message_history(chat_agent_system_message, user_query)

# Prompt the chat_agent
response = run_chat_agent(message_history)

# Update message history
message = [{"role": "assistant", "content": response}]
message_history.append(message)

# Run router_agent
route = run_router_agent(response)


---CHAT AGENT---
{
    "Thought": "I should look up the average weight of a Toy Poodle using the function 'average_dog_weight'.",
    "Action": {"function":"average_dog_weight", "input": "Toy Poodle"},
    "Status": "PAUSE"
}
---ROUTER AGENT---
Status: PAUSE
Route: to_research_agent


In [10]:
def run_research_agent(llm_response):

    print("---RESEARCH AGENT---")

    # Extract the status
    json_response = json.loads(llm_response)
    action_dict = json_response['Action']
    func_to_run = action_dict['function']
    func_input = action_dict['input']

    if func_to_run == "average_dog_weight":
        answer = average_dog_weight(func_input)
    else:
        answer = calculate(func_input)

    print("func_to_run:", func_to_run)
    print("func_arg:", func_input)
    print("Output:", answer)

    return answer



# Example

user_query = "How much does a toy poodle weigh?"

message_history = create_message_history(chat_agent_system_message, user_query)

# Prompt the chat_agent
llm_response = run_chat_agent(message_history)

# Update message history
message = {"role": "assistant", "content": llm_response}
message_history.append(message)

# Run router_agent
route = run_router_agent(llm_response)


if route == "to_research_agent":
    answer = run_research_agent(llm_response)

    # Update message history
    message = {"role": "user", "content": f"Observation: {answer}"}
    message_history.append(message)



---CHAT AGENT---
{
    "Thought": "I should look up the average weight of a Toy Poodle using the average_dog_weight function.",
    "Action": {"function":"average_dog_weight", "input": "Toy Poodle"},
    "Status": "PAUSE"
}
---ROUTER AGENT---
Status: PAUSE
Route: to_research_agent
---RESEARCH AGENT---
func_to_run: average_dog_weight
func_arg: Toy Poodle
Output: a toy poodles average weight is 7 lbs


In [11]:
def run_final_answer_agent(llm_response):

    print("---FINAL ANSWER AGENT---")

    json_response = json.loads(llm_response)
    final_answer = json_response['Answer']

    print("Final answer:", final_answer)

# Run the system

In [12]:
user_query1 = "How much does a Scottish Terrier weigh?"

user_query2 = "How much does a Scottish Terrier and a Border Collie weigh?"

# For this question the model can try to call all
# the functions it needs at the same time.
user_query3 = "What is the average weight of a Scottish Terrier and a Border Collie?"

# The system does not fail when this question is asked.
# The LLM decides by itself what to do.
user_query4 = "Hello. How are you?"


message_history = create_message_history(chat_agent_system_message, user_query4)


for i in range(0,5):

    # Prompt the chat_agent
    llm_response = run_chat_agent(message_history)

    # Update message history
    message = {"role": "assistant", "content": llm_response}
    message_history.append(message)

    # Run router_agent
    route = run_router_agent(llm_response)


    if route == "to_research_agent":

        answer = run_research_agent(llm_response)

        # Update message history
        message = {"role": "user", "content": f"Observation: {answer}"}
        message_history.append(message)

    else:

        run_final_answer_agent(llm_response)

        break



---CHAT AGENT---
{
    "Answer": "I'm fine, thanks.",
    "Status": "DONE"
}
---ROUTER AGENT---
Status: DONE
Route: to_final_answer
---FINAL ANSWER AGENT---
Final answer: I'm fine, thanks.


## Run a chat loop

In [15]:
# Prompting the user for input
#user_input = input("Please enter something: ")

message_history = initialize_message_history(chat_agent_system_message)

while True:

    print()
    print("==========")
    user_input = input("Enter something (or 'q' to quit): ")
    print("==========")

    if user_input.lower() == 'q':
        print("Exiting the loop. Goodbye!")
        break  # Exit the loop

    # Update message history
    message = {"role": "user", "content": user_input}
    message_history.append(message)


    for i in range(0,5):

        # Prompt the chat_agent
        llm_response = run_chat_agent(message_history)

        # Update message history
        message = {"role": "assistant", "content": llm_response}
        message_history.append(message)

        # Run router_agent
        route = run_router_agent(llm_response)


        if route == "to_research_agent":

            answer = run_research_agent(llm_response)

            # Update message history
            message = {"role": "user", "content": f"Observation: {answer}"}
            message_history.append(message)

        else:

            run_final_answer_agent(llm_response)

            break

Enter something (or 'q' to quit):  What is the average weight of a toy poodle and a bulldog?


---CHAT AGENT---
{
    "Thought": "I will first find the average weight of a Toy Poodle, then the average weight of a Bulldog, and finally sum these two weights to get the total.",
    "Action": {
        "function": "average_dog_weight",
        "input": "Toy Poodle"
    },
    "Status": "PAUSE"
}
---ROUTER AGENT---
Status: PAUSE
Route: to_research_agent
---RESEARCH AGENT---
func_to_run: average_dog_weight
func_arg: Toy Poodle
Output: a toy poodles average weight is 7 lbs
---CHAT AGENT---
{
    "Thought": "Now that I have the average weight of a Toy Poodle, I will find the average weight of a Bulldog next.",
    "Action": {
        "function": "average_dog_weight",
        "input": "Bulldog"
    },
    "Status": "PAUSE"
}
---ROUTER AGENT---
Status: PAUSE
Route: to_research_agent
---RESEARCH AGENT---
func_to_run: average_dog_weight
func_arg: Bulldog
Output: An average dog weights 50 lbs
---CHAT AGENT---
{
    "Thought": "I have both the average weight of a Toy Poodle (7 lbs) and an ave

Enter something (or 'q' to quit):  q


Exiting the loop. Goodbye!
